In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import re
import string

In [ ]:
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

Insert the labeling in the CSV file table fake = 0 and true = 1

In [ ]:
true['label'] = 1

In [ ]:
fake['label'] = 0

In [ ]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [ ]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


news = pd.concat([fake, true], axis = 0) use for merge 2 files in one fake and true

In [ ]:
news = pd.concat([fake, true], axis = 0)

In [ ]:
news.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [ ]:
news.tail()

,title,text,subject,date,label
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [ ]:
news.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


news = news.drop(['title', 'subject','date'], axis = 1) use for remove 3 coloumns from data

In [ ]:
news = news.drop(['title', 'subject','date'], axis = 1)

In [ ]:
news.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


Use for reshuffle the date which will be mixed of fake and true

In [ ]:
news = news.sample(frac = 1)

In [ ]:
news.head()

,text,label
9790,WASHINGTON (Reuters) - Efforts to protect the ...,1
20948,Hey Bernie The first step in fighting corrupti...,0
18635,Tim Allen s hit sitcom Last Man Standing has...,0
21210,The drip drip drip of the Left ensuring Christ...,0
4422,(Reuters) - U.S. officials should be respectfu...,1


When you modify a DataFrame (like removing rows, filtering data, etc.), the row numbers (called index) can get messed up or be out of order. reset_index() fixes this problem by giving the DataFrame new index numbers starting from 0, 1, 2, and so on.

In [ ]:
news.reset_index(inplace=True)

In [ ]:
news.head()

,index,text,label
0,9790,WASHINGTON (Reuters) - Efforts to protect the ...,1
1,20948,Hey Bernie The first step in fighting corrupti...,0
2,18635,Tim Allen s hit sitcom Last Man Standing has...,0
3,21210,The drip drip drip of the Left ensuring Christ...,0
4,4422,(Reuters) - U.S. officials should be respectfu...,1


In [ ]:
news.drop(['index'], axis = 1, inplace = True)

In [ ]:
news.head()

,level_0,text,label
0,0,WASHINGTON (Reuters) - Efforts to protect the ...,1
1,1,Hey Bernie The first step in fighting corrupti...,0
2,2,Tim Allen s hit sitcom Last Man Standing has...,0
3,3,The drip drip drip of the Left ensuring Christ...,0
4,4,(Reuters) - U.S. officials should be respectfu...,1


This function cleans text data by removing unnecessary elements like URLs, HTML tags, punctuation, numbers, and newline characters.

In [ ]:
def wordopt(text):
  #convert into lowercase
  text = text.lower()
  #Remove URLs and replacce with nothing
  text = re.sub(r'https?://\S+|www\.\S+', '' ,text)

  #Remove HTML Tags
  text = re.sub(r'<.*?>', '', text)

  #Remove punctuation
  text = re.sub(r'[^\w\s]', '', text)

  #Remove digits
  text = re.sub(r'\d', '', text)

  #Remove newline Characters
  text = re.sub(r'\n', '', text)

  return text

In [ ]:
news['text'] = news['text'].apply(wordopt)

In [ ]:
news['text']

,text
0,washington reuters efforts to protect the gre...
1,hey bernie the first step in fighting corrupti...
2,tim allen s hit sitcom last man standing has...
3,the drip drip drip of the left ensuring christ...
4,reuters us officials should be respectful of ...
...,...
44893,sacramento calif reuters a plan to raise cali...
44894,mark cuban is not a fan on donald trump and he...
44895,special report savvy neocons see hillary clint...
44896,hanoi reuters this isn t like china says vi...


In [ ]:
x = news['text']
y = news['label']

In [ ]:
x

,text
0,washington reuters efforts to protect the gre...
1,hey bernie the first step in fighting corrupti...
2,tim allen s hit sitcom last man standing has...
3,the drip drip drip of the left ensuring christ...
4,reuters us officials should be respectful of ...
...,...
44893,sacramento calif reuters a plan to raise cali...
44894,mark cuban is not a fan on donald trump and he...
44895,special report savvy neocons see hillary clint...
44896,hanoi reuters this isn t like china says vi...


In [ ]:
y

,label
0,1
1,0
2,0
3,0
4,1
...,...
44893,1
44894,0
44895,0
44896,1


Divide a data in 2 parts one is training and other one is testing and the test size is 0.3 means 30% of the data goes to testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [ ]:
x_train.shape

(31428,)

In [ ]:
x_test.shape

(13470,)

In [ ]:
vectorization = TfidfVectorizer()

In [ ]:
xv_train = vectorization.fit_transform(x_train)

In [ ]:
xv_test = vectorization.transform(x_test)

In [ ]:
xv_train

<31428x174917 sparse matrix of type '<class 'numpy.float64'>'
	with 6452238 stored elements in Compressed Sparse Row format>

In [ ]:
xv_test

<13470x174917 sparse matrix of type '<class 'numpy.float64'>'
	with 2728585 stored elements in Compressed Sparse Row format>

In [ ]:
LR = LogisticRegression()

In [ ]:
LR.fit(xv_train, y_train)

LogisticRegression()

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

0.9890126206384559

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7036
           1       0.99      0.99      0.99      6434

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [ ]:
Dtc = DecisionTreeClassifier()

In [ ]:
Dtc.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
predicted_Dtc = Dtc.predict(xv_test)

In [ ]:
Dtc.score(xv_test, y_test)

0.9966592427616926

In [ ]:
print(classification_report(y_test, predicted_Dtc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7036
           1       1.00      1.00      1.00      6434

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(xv_train, y_train)

RandomForestClassifier()

In [ ]:
predicted_rfc = rfc.predict(xv_test)

In [ ]:
rfc.score(xv_test, y_test)

0.9916852264291017

In [ ]:
print(classification_report(y_test, predicted_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7036
           1       0.99      0.99      0.99      6434

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [ ]:
gbc = GradientBoostingClassifier()

In [ ]:
gbc.fit(xv_train, y_train)

GradientBoostingClassifier()

In [ ]:
predicted_gbc = gbc.predict(xv_test)

In [ ]:
gbc.score(xv_test, y_test)

0.996807720861173

In [ ]:
print(classification_report(y_test,predicted_gbc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7036
           1       0.99      1.00      1.00      6434

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [ ]:
def output_label(n):
  if n == 0:
    return "It is a fake news"
  elif n == 1:
    return "It is a true news"
  else:
    return "Invalid news"

In [ ]:
def is_valid_news(news):
    """Check if the input news is valid."""
    if len(news.split()) < 3:  # Too short
        return False
    return True

In [ ]:
def is_news_presentInDS(news, new_data):
  if news not in new_data["text"].values:
    print("Invalid news ❌. This news is not present in the dataset.")
    return False
  else:
    return True

In [ ]:
def manual_testing(news):
  testing_news = {"text": [news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(wordopt)
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)
  pred_lr = LR.predict(new_xv_test)
  predicted_Dtc = Dtc.predict(new_xv_test)
  predicted_rfc = rfc.predict(new_xv_test)
  predicted_gbc = gbc.predict(new_xv_test)

  # 4️⃣ Confidence Check
  prob_lr = max(LR.predict_proba(new_xv_test)[0])  # max probability for LR
  prob_dtc = max(Dtc.predict_proba(new_xv_test)[0])  # max probability for DTC
  prob_rfc = max(rfc.predict_proba(new_xv_test)[0])  # max probability for RFC
  prob_gbc = max(gbc.predict_proba(new_xv_test)[0])  # max probability for GBC

  #Check the probability of news
  if prob_lr < 0.6 and prob_dtc < 0.6 and prob_rfc < 0.6 and prob_gbc < 0.6:
    return print("❌ Invalid news. The model is not confident about this input.")

  return print("\n\n✅ LR Prediction: {} \n✅ DTC Prediction: {} \n✅ RFC Prediction: {} \n✅ GBC Prediction: {}".format(
      output_label(pred_lr[0]),
      output_label(predicted_Dtc[0]),
      output_label(predicted_rfc[0]),
      output_label(predicted_gbc[0])
  ))

In [ ]:
while True:
  new_data = pd.concat([fake, true], axis = 0)
  news_article = str(input("Put your news here! \n"))
  if news_article == 'Exit' or news_article == 'exit' or news_article == 'e':
    print('Thank you for using')
    break
  elif not is_valid_news(news_article):
    print("❌ Invalid news. The input does not look like valid news text.")
  elif is_news_presentInDS(news_article, new_one):
    manual_testing(news_article)
  else:
    print('Invalid input\n')
  print('Type Exit or e for end\n')



✅ LR Prediction: It is a true news 
✅ DTC Prediction: It is a true news 
✅ RFC Prediction: It is a true news 
✅ GBC Prediction: It is a true news
Type Exit or e for end



✅ LR Prediction: It is a fake news 
✅ DTC Prediction: It is a fake news 
✅ RFC Prediction: It is a fake news 
✅ GBC Prediction: It is a fake news
Type Exit or e for end

❌ Invalid news. The input does not look like valid news text.
Type Exit or e for end

Invalid news ❌. This news is not present in the dataset.
Invalid input

Type Exit or e for end



✅ LR Prediction: It is a true news 
✅ DTC Prediction: It is a true news 
✅ RFC Prediction: It is a true news 
✅ GBC Prediction: It is a true news
Type Exit or e for end



✅ LR Prediction: It is a fake news 
✅ DTC Prediction: It is a fake news 
✅ RFC Prediction: It is a fake news 
✅ GBC Prediction: It is a fake news
Type Exit or e for end

Invalid news ❌. This news is not present in the dataset.
Invalid input

Type Exit or e for end

❌ Invalid news. The input does not